# Calculate hit_rate
## Model: EBM models (trained on tabular LDA features)
## Ranking data: random ranking data 
- Input: Top20 recommendation results by ebm_location, ebm_side_info, ebm_extended models
- Hit rate: Evaluated on different benchmarks: knn_lda_ranking_data, random_ranking_data, actual test_interaction data
- Hit rate @5, @10, @20 for each model

In [1]:
import numpy as np
import pandas as pd


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Run this for reproduce
# Gets the current working directory
import os
cwd = os.getcwd()
print("Working directory:", cwd)
# Go up one directory from working directory
os.chdir("..")

Working directory: /Users/anhtth/Library/CloudStorage/OneDrive-UniversityofTwente/2023 UT- THESIS/1-Code/0.cb12_main/nb_recsys_ebm


In [4]:
# Load evaluation data
evaluation_data = pd.read_csv('./nb_recsys_ebm/ranking_data_knn_lda_v2.csv')

In [5]:
evaluation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369098 entries, 0 to 369097
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   UserID   369098 non-null  int64  
 1   JobID    369098 non-null  int64  
 2   label    369098 non-null  float64
 3   City     369098 non-null  float64
 4   State    369098 non-null  float64
 5   Country  369098 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 16.9 MB


In [6]:
evaluation_data.label.value_counts()

0.0    368951
1.0       147
Name: label, dtype: int64

In [7]:
# Load random sample data
ranking_data_random = pd.read_csv('./nb_recsys_ebm/ranking_data_random.csv')
ranking_data_random.label.value_counts()

0    367884
1      3716
Name: label, dtype: int64

In [8]:
models = []
benchmarks = []
hits_5 = []
hits_10 = []
hits_20 = []

## Load recommendation results

In [9]:
ebm_location_recommendations = pd.read_csv('./output_topN_ebm/rec_result_location_random.csv')
ebm_extended_recommendations = pd.read_csv('./output_topN_ebm/rec_result_ebm_extended_random.csv')
ebm_side_info_recommendations = pd.read_csv('./output_topN_ebm/rec_result_side_info_random.csv')

In [10]:
def get_hit_rate_at_k(recommendations, rankings, K):
    total_users = len(recommendations['UserID'].unique())
    hit_count = 0
    # mark_stop = 0
    for user_id in recommendations['UserID'].unique():
        user_recommendations = recommendations[recommendations['UserID'] == user_id]['JobID'].tolist()[:K]
        actual_job_ids = rankings[(rankings['UserID'] == user_id) & (rankings['label'] == 1)]['JobID'].tolist()

        for rec_job_id in user_recommendations:
            if rec_job_id in actual_job_ids:
                # print('hit detail UserID, JobID: ', user_id, rec_job_id)
                hit_count += 1
                break     
        # mark_stop =+1
        # if mark_stop == 2:
        #     break
    print('final hit_count:', hit_count)
    hit_rate = hit_count / total_users
    return hit_rate

# Calculate hit_rate (manual) - Benchmark - random sample data
Positive items: Only consider label 1

In [11]:
benchmark = 'ranking_data_random'

## Model EBM_location

In [12]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_location'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 3716
Model: ebm_location - Hit rate at 5: 1.0
final hit_count: 3716
Model: ebm_location - Hit rate at 10: 1.0
final hit_count: 3716
Model: ebm_location - Hit rate at 20: 1.0
CPU times: user 13.9 s, sys: 112 ms, total: 14.1 s
Wall time: 14.1 s


In [13]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

## Model: ebm_side_info

In [14]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_side_info'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20
hit_20 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 516
Model: ebm_side_info - Hit rate at 5: 0.13885898815931108
final hit_count: 910
Model: ebm_side_info - Hit rate at 10: 0.2448869752421959
final hit_count: 1343
Model: ebm_side_info - Hit rate at 20: 0.36141011840688914
CPU times: user 13.9 s, sys: 98.2 ms, total: 14 s
Wall time: 14.1 s


In [15]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

## Model: ebm_extended

In [16]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_extended'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = ranking_data_random, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 3305
Model: ebm_extended - Hit rate at 5: 0.8893972012917115
final hit_count: 3309
Model: ebm_extended - Hit rate at 10: 0.8904736275565124
final hit_count: 3366
Model: ebm_extended - Hit rate at 20: 0.9058127018299247
CPU times: user 13.6 s, sys: 90 ms, total: 13.7 s
Wall time: 13.8 s


In [17]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

## Calculate hit_rate (manual) - Benchmark - evaluation data
Positive items: Only consider label 1

In [18]:
benchmark = 'ranking_knn_lda'

### Model: ebm_location

In [19]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_location'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 0
Model: ebm_location - Hit rate at 5: 0.0
final hit_count: 0
Model: ebm_location - Hit rate at 10: 0.0
final hit_count: 0
Model: ebm_location - Hit rate at 20: 0.0
CPU times: user 13.1 s, sys: 89.7 ms, total: 13.2 s
Wall time: 13.2 s


In [20]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: ebm_side_info

In [21]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_side_info'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20
hit_20 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 0
Model: ebm_side_info - Hit rate at 5: 0.0
final hit_count: 0
Model: ebm_side_info - Hit rate at 10: 0.0
final hit_count: 0
Model: ebm_side_info - Hit rate at 20: 0.0
CPU times: user 13.1 s, sys: 91.4 ms, total: 13.2 s
Wall time: 13.2 s


In [22]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: ebm_extended

In [23]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_extended'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = evaluation_data, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k}: {hit_20}")

final hit_count: 0
Model: ebm_extended - Hit rate at 5: 0.0
final hit_count: 0
Model: ebm_extended - Hit rate at 10: 0.0
final hit_count: 0
Model: ebm_extended - Hit rate at 20: 0.0
CPU times: user 13.2 s, sys: 92.3 ms, total: 13.3 s
Wall time: 13.3 s


In [24]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

# Calculate hit_rate (manual) - Evaluation data: test_interaction data

In [25]:
benchmark = 'actual_test_interaction'

In [26]:
# Load all interaction data (LDA tabular with side information)
train_data_extended = pd.read_csv('./xai_recsys/train_data_extended.csv')
test_data_extended = pd.read_csv('./xai_recsys/test_data_extended.csv')

In [27]:
# Get all interaction data
select_cols = ['UserID', 'JobID', 'label','Split']
actual_interaction_train = train_data_extended[select_cols]
actual_interaction_test = test_data_extended[select_cols]
actual_interaction_df = pd.concat([actual_interaction_train,actual_interaction_test])

In [28]:
len(actual_interaction_train), len(actual_interaction_test)

(563889, 15736)

### Model: EBM_location

In [29]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_location'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_20}")

final hit_count: 3716
Model: ebm_location - Hit rate at 5 - evaluated on actual interaction (test): 1.0
final hit_count: 3716
Model: ebm_location - Hit rate at 10 - evaluated on actual interaction (test): 1.0
final hit_count: 3716
Model: ebm_location - Hit rate at 20 - evaluated on actual interaction (test): 1.0
CPU times: user 8.23 s, sys: 32.1 ms, total: 8.26 s
Wall time: 8.41 s


In [30]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: EBM_side_info

In [31]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_side_info'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_20}")

final hit_count: 516
Model: ebm_side_info - Hit rate at 5 - evaluated on actual interaction (test): 0.13885898815931108
final hit_count: 910
Model: ebm_side_info - Hit rate at 10 - evaluated on actual interaction (test): 0.2448869752421959
final hit_count: 1343
Model: ebm_side_info - Hit rate at 20 - evaluated on actual interaction (test): 0.36141011840688914
CPU times: user 8.28 s, sys: 35 ms, total: 8.31 s
Wall time: 8.36 s


In [32]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: EBM_extended

In [33]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_extended'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_test, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (test): {hit_20}")

final hit_count: 3305
Model: ebm_extended - Hit rate at 5 - evaluated on actual interaction (test): 0.8893972012917115
final hit_count: 3309
Model: ebm_extended - Hit rate at 10 - evaluated on actual interaction (test): 0.8904736275565124
final hit_count: 3366
Model: ebm_extended - Hit rate at 20 - evaluated on actual interaction (test): 0.9058127018299247
CPU times: user 8.29 s, sys: 34.2 ms, total: 8.33 s
Wall time: 8.34 s


In [34]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

## Calculate hit_rate (manual) - Evaluation data: all interaction data

In [35]:
benchmark = 'all_interaction_data'

### Model: EBM_location

In [36]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_location'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_location_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_20}")

final hit_count: 3716
Model: ebm_location - Hit rate at 5 - evaluated on actual interaction (all): 1.0
final hit_count: 3716
Model: ebm_location - Hit rate at 10 - evaluated on actual interaction (all): 1.0
final hit_count: 3716
Model: ebm_location - Hit rate at 20 - evaluated on actual interaction (all): 1.0
CPU times: user 17 s, sys: 102 ms, total: 17.1 s
Wall time: 17.2 s


In [37]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: side_info

In [38]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_side_info'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_side_info_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_20}")

final hit_count: 516
Model: ebm_side_info - Hit rate at 5 - evaluated on actual interaction (all): 0.13885898815931108
final hit_count: 910
Model: ebm_side_info - Hit rate at 10 - evaluated on actual interaction (all): 0.2448869752421959
final hit_count: 1343
Model: ebm_side_info - Hit rate at 20 - evaluated on actual interaction (all): 0.36141011840688914
CPU times: user 17.3 s, sys: 101 ms, total: 17.4 s
Wall time: 17.4 s


In [39]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

### Model: EBM - extended

In [40]:
%%time
# Calculate hit rate at K (precision at K)
model = 'ebm_extended'
k = 5  # Set the desired value of K
hit_5 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_5}")

k = 10  # Set the desired value of K
hit_10 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_10}")

k = 20  # Set the desired value of K
hit_20 = get_hit_rate_at_k(recommendations = ebm_extended_recommendations, 
                                   rankings = actual_interaction_df, 
                                   K=k)
print(f"Model: {model} - Hit rate at {k} - evaluated on actual interaction (all): {hit_20}")

final hit_count: 3305
Model: ebm_extended - Hit rate at 5 - evaluated on actual interaction (all): 0.8893972012917115
final hit_count: 3309
Model: ebm_extended - Hit rate at 10 - evaluated on actual interaction (all): 0.8904736275565124
final hit_count: 3366
Model: ebm_extended - Hit rate at 20 - evaluated on actual interaction (all): 0.9058127018299247
CPU times: user 17.1 s, sys: 111 ms, total: 17.2 s
Wall time: 17.2 s


In [41]:
models.append(model)
benchmarks.append(benchmark)
hits_5.append(hit_5)
hits_10.append(hit_10)
hits_20.append(hit_20)

# Summary

In [42]:
len(models), len(benchmarks), len(hits_5), len(hits_10), len(hits_20)

(12, 12, 12, 12, 12)

In [43]:
ebm_hit = pd.DataFrame(columns = ["model","hit_5", "hit_10", "hit_20", 'benchmark'])
ebm_hit.model = models
ebm_hit.benchmark = benchmarks
ebm_hit.hit_5 = hits_5
ebm_hit.hit_10 = hits_10
ebm_hit.hit_20 = hits_20

In [44]:
ebm_hit

,model,hit_5,hit_10,hit_20,benchmark
0,ebm_location,1.000000,1.000000,1.000000,ranking_data_random
1,ebm_side_info,0.138859,0.244887,0.361410,ranking_data_random
2,ebm_extended,0.889397,0.890474,0.905813,ranking_data_random
3,ebm_location,0.000000,0.000000,0.000000,ranking_knn_lda
4,ebm_side_info,0.000000,0.000000,0.000000,ranking_knn_lda
5,ebm_extended,0.000000,0.000000,0.000000,ranking_knn_lda
6,ebm_location,1.000000,1.000000,1.000000,actual_test_interaction
7,ebm_side_info,0.138859,0.244887,0.361410,actual_test_interaction
8,ebm_extended,0.889397,0.890474,0.905813,actual_test_interaction
9,ebm_location,1.000000,1.000000,1.000000,all_interaction_data


In [45]:
ebm_hit.to_csv('./nb_recsys_ebm/ebm_hit_random.csv', header=True, index=False)

# Data-centric explanation
- Count number of JobID for positive interaction in test data

In [ ]:
summary_test_interaction = actual_interaction_test.groupby('UserID').agg(Count_JobID=('JobID', 'count')).reset_index()

In [ ]:
summary_test_interaction.sort_values(by = 'Count_JobID', ascending=False)

In [ ]:
summary_test_interaction['Count_JobID'].value_counts()

In [ ]:
summary_positive_test_interaction = actual_interaction_test[actual_interaction_test.label==1].groupby('UserID').agg(Count_JobID=('JobID', 'count')).reset_index()

In [ ]:
summary_positive_test_interaction.sort_values(by = 'Count_JobID', ascending=False)

In [ ]:
summary_positive_test_interaction['Count_JobID'].value_counts()